In [ ]:
# -*- coding: utf-8 -*-
"""Untitled0.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1yQ-wrRfeQwg7IcdmApJdmASptqeL1Qx0
"""

# from google.colab import drive
# drive.mount('/content/gdrive')

# Commented out IPython magic to ensure Python compatibility.
# %tensorflow_version 1.x

import tensorflow
tensorflow.__version__

import tensorflow.keras as keras
keras.__version__

# !ls '/content/gdrive/My Drive/Datasets/kannada-mnist'

# !cp -r '/content/gdrive/My Drive/Datasets/kannada-mnist' ./

# !ls ./kannada-mnist/

import pandas as pd
from keras.layers import *
from keras.models import *
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras import losses
from keras import optimizers
from sklearn.model_selection import train_test_split

train = pd.read_csv('../input/Kannada-MNIST/train.csv')
test = pd.read_csv('../input/Kannada-MNIST/test.csv')
Dig_MNIST = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv')

test.tail()

import numpy as np

x = train.iloc[:, 1:].values
y = np.array([ [ 1 if (i == label) else 0 for i in range(10)] for label in train.iloc[:, :1].values ])
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=10000, random_state=1997)

inputs = Input(shape=(784,))
net = BatchNormalization()(inputs)
net = Dense(20, activation=None)(net)
net = Activation('elu')(net)
net = Dropout(rate=0.3)(net)
net = Dense(20, activation=None)(net)
net = Activation('elu')(net)
prediction = Dense(10, activation='softmax')(net)

model = Model(inputs=[inputs], output=[prediction])
model.compile('RMSprop', losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()

(x_train.shape, y_train.shape)

model.fit(
    x_train, y_train,
    batch_size=16,
    epochs=5,
    verbose=2,
    validation_data=(x_valid, y_valid),
    callbacks=[
      CSVLogger('log.csv'),
      ModelCheckpoint('model.h5', save_best_only=True),
    ]
)

x_test=test.drop('id', axis=1).iloc[:,:].values
x_test.shape

pred_probas = model.predict(x_test - 0.5, batch_size=16)
prediction = pred_probas.argmax(axis=1)

submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submission['label'] = prediction
submission.to_csv("submission.csv", index=False)
submission.tail()